In [1]:
from datasets import load_dataset

dataset = load_dataset("umarbutler/open-australian-legal-qa")
documents_dataset = load_dataset("umarbutler/open-australian-legal-corpus")

README.md:   0%|          | 0.00/9.11k [00:00<?, ?B/s]

qa.jsonl:   0%|          | 0.00/13.5M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

corpus.jsonl:   0%|          | 0.00/9.31G [00:00<?, ?B/s]

Generating corpus split:   0%|          | 0/229122 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'text', 'prompt', 'source'],
        num_rows: 2124
    })
})

In [7]:
from tqdm import tqdm

corpus_dict = {item['version_id']: item['text'] for item in tqdm(documents_dataset['corpus'])}

100%|██████████| 229122/229122 [00:31<00:00, 7347.17it/s]


In [4]:
import json

print(json.dumps(dataset['train'][0], indent=2))

{
  "question": "In the case of Nasr v NRMA Insurance [2006] NSWSC 1018, why was the plaintiff's appeal lodged out of time?",
  "answer": "In Nasr v NRMA Insurance [2006] NSWSC 1018, the plaintiff's appeal was lodged out of time because the summons was filed on 8 June 2006, seven months after the decision of the Local Court was made on 4 October 2005. No explanation was provided for this delay.",
  "text": "Question: In the case of Nasr v NRMA Insurance [2006] NSWSC 1018, why was the plaintiff's appeal lodged out of time?\nAnswer: In Nasr v NRMA Insurance [2006] NSWSC 1018, the plaintiff's appeal was lodged out of time because the summons was filed on 8 June 2006, seven months after the decision of the Local Court was made on 4 October 2005. No explanation was provided for this delay.",
  "prompt": "# Snippet\nThe snippet from an Australian legal document from which you must synthesise a question and answer is provided below.\n<document_metadata>\n<document_title>Nasr v NRMA Insurance 

In [ ]:
import spacy
from datasets import DatasetDict
import os

num_proc=os.cpu_count()

from more_itertools import windowed
nlp = spacy.load("en_core_web_sm")

def chunk_document(document):
    doc = nlp(document)
    sentences = [sent.text for sent in doc.sents]
    return sentences

def process_record(record):
    document = record['context']
    gold_text = record['answer']
    source = record['source']
    citations = [[source['version_id'], ]]
    sentences = chunk_document(document)
    oracle_documents_passages = []
    for sentence in windowed(sentences, 5, fillvalue="", step=2):
        section = " ".join([s for s in sentence if (s is not None and len(s.strip()) > 0)])
        oracle_documents_passages.append([document_id, section])
    
    return {
        'previous_text': record['question'],
        'gold_text': gold_text,
        'citations': citations,
        'oracle_documents_passages': oracle_documents_passages,
    }

dataset_train = dataset['train'].select(range(1000))
dataset_test = dataset['test']

dataset = DatasetDict({
    'train': dataset_train,
    'test': dataset_test,
})
dataset = dataset.map(process_record, num_proc=num_proc)